### Web scraping using Selenium

For this project I'm working on scraping jobs data from Glassdoor.com

To start my code I have used an already existing jupyter notebook 'https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb' and I have changed different parts of it and I have modified all the XPATHS as the HTML code of the Glassdoor web page have changed a lot since the first code was written. 
I have started this project 2 days ago and my code is not ready to extract a huge amount of data but **you can see at the end of this code what the desired dataframe will look like**. I am still working on scrolling the Glassroom page to show the next hidden jobs.

In [4]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

In [15]:
def get_jobs_dataframe(keyword, num_jobs, verbose, path, slp_time):
    
    '''Scraping "Glassdoor" to gather jobs as a dataframe
    '''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1000, 1000)
    url = "https://www.glassdoor.com/Job/jobs.htm?context=Jobs&suggestCount=0&suggestChosen=false&clickSource=searchBox&typedKeyword=" + keyword + "&sc.keyword=" + keyword
    driver.get(url)
    jobs = []
    j=0
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.
 
        #Wait until the webpage is loaded
        time.sleep(slp_time)
        
        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_xpath('.//span[@class="SVGInline modal_closeIcon"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        #Going through each job in this page
        job_buttons = driver.find_elements_by_xpath('//a[@class="jobLink css-1rd3saf eigr9kq2"]') 
        for job_button in job_buttons: 
            general_info_dic = {'Company name':-1, 'Location':-1, 'Job title':-1, 'Job description':-1, 'Salary estimate':-1, 'Rating':-1}
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            driver.implicitly_wait(3) # seconds
            if len(jobs) >= num_jobs:
                break
                
            actions = ActionChains(driver)
            actions.move_to_element(job_button)
            actions.perform()
            driver.implicitly_wait(3) # seconds
            actions.click()
            actions.perform()

            try:
                driver.find_element_by_xpath('//span[@class="SVGInline modal_closeIcon"]').click()  #clicking to the X.
            except NoSuchElementException:
                pass
            
            j+=1
            print(j)
            collected_successfully = False
            while not collected_successfully:
                try:
                    general_info_dic['Company name'] = driver.find_element_by_xpath('//div[@class="css-87uc0g e1tk4kwz1"]').text.split("\n")[0]
                    general_info_dic['Location'] = driver.find_element_by_xpath('//div[@class="css-56kyx5 e1tk4kwz5"]').text
                    general_info_dic['Job title'] = driver.find_element_by_xpath('//div[@class="css-1vg6q84 e1tk4kwz4"]').text
                    general_info_dic['Job description'] = driver.find_element_by_xpath('//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)


            try:
                general_info_dic['Salary estimate'] = driver.find_element_by_xpath('//span[@class="css-56kyx5 css-16kxj2j e1wijj242"]').text
            except NoSuchElementException:
                general_info_dic['Salary estimate'] = -1 #You need to set a "not found value. It's important."
            try:
                general_info_dic['Rating'] = driver.find_element_by_xpath('//span[@class="css-1m5m32b e1tk4kwz2"]').text
            except NoSuchElementException:
                general_info_dic['Rating'] = -1 

            
            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(general_info_dic['Job title']))
                print("Salary Estimate: {}".format(general_info_dic['Salary estimate']))
                print("Job Description: {}".format(general_info_dic['Job description'][:600]))
                print("Rating: {}".format(general_info_dic['Rating']))
                print("Company Name: {}".format(general_info_dic['Company name']))
                print("Location: {}".format(general_info_dic['Location']))


            #Going to the Company tab
            try:
                company_overview_dic = {'Size':-1, 'Founded':-1, 'Type':-1, 'Industry':-1, 'Sector':-1, 'Revenue':-1}
                target = driver.find_element_by_xpath('//div[@data-item="tab" and @data-tab-type="overview"]')
                target.click()

                provided_titles = []
                provided_contents = []
                try:
                    provided_titles = driver.find_elements_by_xpath('//span[@class="css-1taruhi e1pvx6aw1"]')         
                    provided_contents = driver.find_elements_by_xpath('//span[@class="css-i9gxme e1pvx6aw2"]')
                except:
                    pass
                if len (provided_titles) > 0:
                    for i in range(len(provided_titles)):
                        title = provided_titles[i].text
                        content = provided_contents[i].text
                        company_overview_dic[str(title)] = content

                Size = company_overview_dic['Size']
                Founded = int(company_overview_dic['Founded'])
                Type = company_overview_dic['Type']
                Industry = company_overview_dic['Industry']
                Sector = company_overview_dic['Sector']
                Revenue = company_overview_dic['Revenue']
            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                print("some job postings do not have the Company tab")


            if verbose:
                print("Size: {}".format(Size))
                print("Founded: {}".format(Founded))
                print("Type: {}".format(Type))
                print("Industry: {}".format(Industry))
                print("Sector: {}".format(Sector))
                print("Revenue: {}".format(Revenue))
                print("//////////////////////////////////////////////")

            jobs.append({"Job Title" : general_info_dic['Job title'],
            "Salary Estimate" : general_info_dic['Salary estimate'],
            "Job Description" : general_info_dic['Job description'],
            "Rating" : general_info_dic['Rating'],
            "Company Name" : general_info_dic['Company name'],
            "Location" : general_info_dic['Location'],
            "Size" : Size,
            "Founded" : Founded,
            "Type of ownership" : Type,
            "Industry" : Industry,
            "Sector" : Sector,
            "Revenue" : Revenue})
            #add job to jobs

            #Test for the "Sign Up" prompt and get rid of it.
            try:
                driver.find_element_by_xpath('//span[@class="SVGInline modal_closeIcon"]').click()  #clicking to the X.
            except NoSuchElementException:
                pass

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('//span[@class="SVGInline"]').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
        time.sleep(3)  
        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_xpath('//span[@class="SVGInline modal_closeIcon"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [16]:
path = "C:\Program Files (x86)\chromedriver"
df = get_jobs_dataframe("Data Scientist", 5, True, path, 20)

Progress: 0/5
1
Job Title: Data Scientist
Salary Estimate: -1
Job Description: XL Fleet seeks an experienced Data Scientist to support the data science activity for hybrid electric, plugin hybrid electric and full electric vehicle product lines.

About XL Fleet:

XL Fleet is on a mission to electrify the global fleet of commercial vehicles. We know this mission can only be successfully with the right talent.

XL is a high-growth commercial vehicle technology company. We are developing and producing cutting-edge technology to convert conventional vehicles into hybrids, plug-in hybrids and zero CO2 drive systems to reduce fuel consumption and to generate a compelling retur
Rating: 5.0
Company Name: XL Fleet
Location: Detroit, MI
Size: 51 to 200 Employees
Founded: 2012
Type: Company - Public
Industry: Transportation Equipment Manufacturing
Sector: Manufacturing
Revenue: $10 to $25 million (USD)
//////////////////////////////////////////////
Progress: 1/5
2
Job Title: Principal Data Scient

In [17]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,-1,XL Fleet seeks an experienced Data Scientist t...,5.0,XL Fleet,"Detroit, MI",51 to 200 Employees,2012,Company - Public,Transportation Equipment Manufacturing,Manufacturing,$10 to $25 million (USD)
1,Principal Data Scientist,-1,"Working out of our Chevy Chase, MD/Washington ...",3.5,GEICO,"Chevy Chase, MD",10000+ Employees,1936,Subsidiary or Business Segment,Insurance Carriers,Insurance,$10+ billion (USD)
2,Data Scientist,-1,"Secure our Nation, Ignite your Future\nOvervie...",-1,ManTech International Corporation,"Alexandria, VA",10000+ Employees,1936,Subsidiary or Business Segment,Insurance Carriers,Insurance,$10+ billion (USD)
3,Scientist,-1,"Job Description\n\nOur client, a leading CPG c...",2.2,Sterling Hoffman,"Deerfield Beach, FL",201 to 500 Employees,-1,Company - Private,Staffing & Outsourcing,Business Services,Less than $1 million (USD)
4,Business Intelligence Data Engineer,-1,Welcome to Foresight Mental Health! We are a m...,4.6,Foresight Mental Health,"Austin, TX",201 to 500 Employees,2018,Company - Private,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable
